In [1]:
import numpy as np
import pandas as pd

#dataset import
dataset = pd.read_csv("train.csv") 
dataset.head(3)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2


In [2]:
#načítanie dát pomocou pandas knižnice
X = dataset.iloc[:,:20].values #informácie o zariadení okrem cenovej skupiny
y = dataset.iloc[:,20:21].values #posledný stĺpec, cenová skupina
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() 
X = sc.fit_transform(X) #úprava údajov na štandardizovanú hodnotu.
X

array([[-0.90259726, -0.9900495 ,  0.83077942, ..., -1.78686097,
        -1.00601811,  0.98609664],
       [-0.49513857,  1.0100505 , -1.2530642 , ...,  0.55964063,
         0.99401789, -1.01409939],
       [-1.5376865 ,  1.0100505 , -1.2530642 , ...,  0.55964063,
         0.99401789, -1.01409939],
       ...,
       [ 1.53077336, -0.9900495 , -0.76274805, ...,  0.55964063,
         0.99401789, -1.01409939],
       [ 0.62252745, -0.9900495 , -0.76274805, ...,  0.55964063,
         0.99401789,  0.98609664],
       [-1.65833069,  1.0100505 ,  0.58562134, ...,  0.55964063,
         0.99401789,  0.98609664]])

In [3]:
from sklearn.preprocessing import OneHotEncoder 
ohe = OneHotEncoder() #import OHN
y = ohe.fit_transform(y).toarray() # premenenie hodnôt y z int na bool array
y

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [4]:
from sklearn.model_selection import train_test_split 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1) # rozdelenie dát na train,test pre X y. test_size urcite podiel training(90%) a test (10%)

In [5]:
import keras # import neurónovej sieti Keras
from keras.models import Sequential #import sekvenčného modelu 
from keras.layers import Dense #Dense (plne prepojené) vrstvy , spája všetky input hodnoty s output hudnotamy na každej vrstve

# Neural network
model = Sequential() # sekvenčný model NS
#add 2 skryté vrstvy o 20 a 12 dimenzií Dense(20,...) a Dense(12,...)
model.add(Dense(20, input_dim=20, activation="relu")) # určuje vstupnú dimenziu 20 (dáta) a aktivačnú funkciu relu
model.add(Dense(12, activation="relu")) # podobne určuje vrstvu s 12 dimenziami, keďže ide o sekvenčný model Kerasu, automaticky ho prevezme z 1.
model.add(Dense(4, activation="softmax")) # output je nastaveny na 4, pretože cenové dĺžky sú od 1 po 4, teda 4 objekty na klasifikovanie
model.summary()
# softmax je aktivačná funkcia ktorá berie výstup poslednej vrstvy viactriednej klasifikačnej neurónovej sieti
#vezme exponenty každého výstupu a potom normalizuje každé číslo súčtom týchto exponentov, takže celý výstupný vektor sa bude rovnať 1 - všetky pravdepodobnosti by sa mali po súčte rovnať 1

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 12)                252       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 52        
Total params: 724
Trainable params: 724
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#Adam je optimalizačný algoritmus, ktorý sa dá použiť na aktualizáciu iteratívnych váh v sieti na základe trénovacích dát.
#Metrics určuje na základe čoho sa bude presnosť siete merať 

In [ ]:
history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=110, batch_size=100)
# na preprivpravený model sa zavolá funkcia fit ktorá berie ako vstupné parametre dáta z datasetu, epochs = počet iterácií a batch_size kt predstavuje aké množstvo údajov sa spracováva naraz. 

Train on 1800 samples, validate on 200 samples
Epoch 1/110
1800/1800 [==============================] - 1s 299us/step - loss: 1.6270 - accuracy: 0.1644 - val_loss: 1.5687 - val_accuracy: 0.1650
Epoch 2/110
1800/1800 [==============================] - 0s 51us/step - loss: 1.4988 - accuracy: 0.2150 - val_loss: 1.4755 - val_accuracy: 0.2250
Epoch 3/110
1800/1800 [==============================] - 0s 59us/step - loss: 1.4198 - accuracy: 0.2594 - val_loss: 1.4134 - val_accuracy: 0.2550
Epoch 4/110
1800/1800 [==============================] - 0s 46us/step - loss: 1.3643 - accuracy: 0.3056 - val_loss: 1.3645 - val_accuracy: 0.3450
Epoch 5/110
1800/1800 [==============================] - 0s 52us/step - loss: 1.3172 - accuracy: 0.3561 - val_loss: 1.3165 - val_accuracy: 0.3700
Epoch 6/110
1800/1800 [==============================] - 0s 40us/step - loss: 1.2702 - accuracy: 0.4078 - val_loss: 1.2671 - val_accuracy: 0.4000
Epoch 7/110
1800/1800 [==============================] - 0s 60us/step - loss

In [ ]:
y_pred = model.predict(X_test)
#vygeneruje predikcie na zaklade vstupnych dat
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i])) #argmax vráti pozíciu najväčšej hodnoty (teda prekonvertuje na int z OHT)
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test) # porovnáva hodnoty a vyhodnotí na kolko % sa výsledky zhodujú 
print("Presnosť(%):", a*100)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Presnosť modelu")
plt.ylabel("Presnosť")
plt.xlabel("Epoch")
plt.legend(["Train", "Test"], loc="upper left")
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('chybovosť')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()